In [1]:
import os

In [2]:
%pwd

'c:\\Users\\tusha\\OneDrive\\Desktop\\Data-Science-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\tusha\\OneDrive\\Desktop\\Data-Science-Project'

In [ ]:
## confiq_entity


from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path 

In [6]:
from src.datascience.constant import *
from src.datascience.utils.comman import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
from urllib.request import urlretrieve
import os
import zipfile
from src.datascience import logger

class DataIngestion:
    def __init__(self, config):
        self.config = config

    ## downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with header: {header}")
        else:
            logger.info(f"{self.config.local_data_file} already exists")

    ## unzipping the file
    def unzip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    raise e

[2025-02-11 17:48:27,525: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2025-02-11 17:48:27,530: INFO: comman: yaml file: params.yaml loaded successfully]
[2025-02-11 17:48:27,535: INFO: comman: yaml file: schema.yaml loaded successfully]
[2025-02-11 17:48:27,536: INFO: comman: created directory at: artifacts]
[2025-02-11 17:48:27,538: INFO: comman: created directory at: artifacts/data_ingestion]
[2025-02-11 17:48:29,793: INFO: 753959703: artifacts/data_ingestion/data.zip downloaded with header: Connection: close
Content-Length: 320898
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "37eed439a1470fcaf01ea1b18e0adf275bb0a75653b9b6046fd14f8b03a0b22d"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F753:10321D:FA7B7:162A0D:67AB4014
Accept-Ranges: bytes
Date: Tue, 11 Feb 2025